<a href="https://colab.research.google.com/github/ucfilho/marquesgabi_mar_2020/blob/master/04_ANN_fotos_testa_variaveis_duas_classes_mar_13_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# rede neural com duas classes grao=2 e nao grao=1+3

In [0]:
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import math
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 
from mpl_toolkits.mplot3d import Axes3D, axes3d
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import matplotlib
import re
%matplotlib inline

In [2]:
!git clone https://github.com/ucfilho/marquesgabi_mar_2020 #clonar do Github
%cd marquesgabi_mar_2020

fatal: destination path 'marquesgabi_mar_2020' already exists and is not an empty directory.
/content/marquesgabi_mar_2020


In [0]:
#!ls

In [0]:
#@title Digite os parametros { run: "auto", vertical-output: true }
#@markdown Enter neurons hidden layer
N_Hidden=22  #@param {type:"number"}
#@markdown Activation function

Ativa='tanh'#@param ["tanh", "identity", "logistic","relu"] {allow-input: true}

In [0]:
# funcao normatiza dados
def Normatiza(x):
    strings=list(x)
    for i in strings:
        max_x=x[i].max()
        min_x=x[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=(x[i]-a)/b
    return x

In [0]:
# funcao retorna os dados a forma original
def Original(x,x_old):
    strings=list(x)
    for i in strings:
        max_x=x_old[i].max()
        min_x=x_old[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=x[i]*b+a
    return x

In [7]:

df_contraste= pd.read_csv('Contraste_Completo_mar_2020.csv') 
#df_contraste= np.delete(df_contraste, 0, axis=1) # delete first colunm

Nomes=list(df_contraste.columns)
del df_contraste['Unnamed: 0'] # delete first colunm
print(Nomes)
print(df_contraste.head())
print(df_contraste.shape)

['Unnamed: 0', 'Type', 'ASM', 'constrast', 'correl', 'variance', 'inv diff mom', 'sum aveg', 'sum var', 'sum entropy', 'entropy', 'dif var', 'dif entropy', 'IMC1', 'IMC2', 'colors']
   Type      ASM  constrast   correl  ...  dif entropy     IMC1     IMC2  colors
0     1  0.06250  220.45629  0.33330  ...      2.70205 -0.66373  0.98529  0.5700
1     1  0.06265   35.93913  0.15288  ...      2.38193 -0.63710  0.98340  0.4250
2     1  0.05668   94.66258  0.35417  ...      2.40839 -0.67320  0.98987  0.4475
3     1  0.05367  364.05229  0.24475  ...      2.94304 -0.73084  0.99238  0.4875
4     1  0.05144  196.93954  0.39782  ...      2.84978 -0.77013  0.99737  0.5350

[5 rows x 15 columns]
(540, 15)


In [8]:
df_fracionado= pd.read_csv('Fracionado_Completo_mar_2020.csv') 
#df_fracionado= np.delete(df_fracionado, 0, axis=1) # delete first colunm
del df_fracionado['Unnamed: 0'] # delete first colunm
del df_fracionado['Type'] # delete first colunm
Nomes_fracionado=list(df_fracionado.columns)
print(df_fracionado.head())
print(df_fracionado.shape)

   N_peaks  Media_proem        Area  ...      Mode       Mean         Sd
0        2     21.73958  1024.86111  ...  15.80556   66.99632   68.46875
1        1     13.34722  1140.79398  ...  10.85417   86.34600   95.53923
2        2     50.77083  2055.15972  ...  30.75000  130.03922   85.50475
3        2    196.14815  4612.20139  ...  13.15278  285.03799  368.87534
4        2    248.66551  5680.38889  ...  13.53472  347.03758  297.34679

[5 rows x 10 columns]
(540, 10)


In [9]:
#print(Nomes_contraste)
print(Nomes_fracionado)

['N_peaks', 'Media_proem', 'Area', 'Width_peaks', 'Width_peaks_max', 'Width_peaks_min', 'Median', 'Mode', 'Mean', 'Sd']


In [10]:
#ANN_Completo é o dataframe que tem todas as variaveis juntas

ANN_Completo = pd.concat([df_contraste, df_fracionado], axis=1)
print(ANN_Completo.head())
print(ANN_Completo.shape)

   Type      ASM  constrast   correl  ...     Median      Mode       Mean         Sd
0     1  0.06250  220.45629  0.33330  ...   41.70833  15.80556   66.99632   68.46875
1     1  0.06265   35.93913  0.15288  ...   34.56250  10.85417   86.34600   95.53923
2     1  0.05668   94.66258  0.35417  ...  116.78472  30.75000  130.03922   85.50475
3     1  0.05367  364.05229  0.24475  ...   70.88889  13.15278  285.03799  368.87534
4     1  0.05144  196.93954  0.39782  ...  341.29167  13.53472  347.03758  297.34679

[5 rows x 25 columns]
(540, 25)


In [11]:
#Aqui iremos escolher quais variáveis iremos utilizar na ANN

Colunms=['N_peaks', 'Media_proem', 'Area', 'Width_peaks', 'Width_peaks_max', 'Width_peaks_min', 'Median', 'Mode','Mean', 'Sd']

#Colunms=['N_peaks', 'Media_proem', 'Area', 'Width_peaks', 
         # 'Width_peaks_max', 'Width_peaks_min', 'Median', 
          #'Mode', 'Mean', 'Sd']

#'ASM', 'constrast', 'correl', 'variance', 'inv diff mom', 'sum aveg', 'sum var', 'sum entropy', 'entropy', 'dif var', 'dif entropy', 'IMC1', 'IMC2', 'colors'
#'N_peaks', 'Media_proem', 'Area', 'Width_peaks', 'Width_peaks_max', 'Width_peaks_min', 'Median', 'Mode', 'Mean', 'Sd'


del df_contraste['Type'] # delete first colunm

Nomes_contraste=list(df_contraste.columns)
for ii in Nomes_contraste:
  Colunms.append(ii)
#X=df.iloc[:,2:12]
#y=df.iloc[:,1].astype(int)
X=ANN_Completo[Colunms]
print(X)
#y=df['Type'].astype(int)
y=ANN_Completo['Type']
print(y)

     N_peaks  Media_proem         Area  ...     IMC1     IMC2  colors
0          2     21.73958   1024.86111  ... -0.66373  0.98529  0.5700
1          1     13.34722   1140.79398  ... -0.63710  0.98340  0.4250
2          2     50.77083   2055.15972  ... -0.67320  0.98987  0.4475
3          2    196.14815   4612.20139  ... -0.73084  0.99238  0.4875
4          2    248.66551   5680.38889  ... -0.77013  0.99737  0.5350
..       ...          ...          ...  ...      ...      ...     ...
535        2      9.60880   1926.58796  ... -0.78025  0.99750  0.5000
536        2    330.73032  15839.67593  ... -0.76151  0.99493  0.5550
537        2     54.44907   1677.80787  ... -0.60750  0.97856  0.6150
538        2     14.74306    662.65509  ... -0.69903  0.99119  0.4900
539        3     19.56096   1105.09722  ... -0.75957  0.99643  0.4625

[540 rows x 24 columns]
0      1
1      1
2      1
3      1
4      1
      ..
535    3
536    3
537    3
538    3
539    3
Name: Type, Length: 540, dtype: int6

In [12]:
print(Colunms)

['N_peaks', 'Media_proem', 'Area', 'Width_peaks', 'Width_peaks_max', 'Width_peaks_min', 'Median', 'Mode', 'Mean', 'Sd', 'ASM', 'constrast', 'correl', 'variance', 'inv diff mom', 'sum aveg', 'sum var', 'sum entropy', 'entropy', 'dif var', 'dif entropy', 'IMC1', 'IMC2', 'colors']


In [13]:

# separa os dados variaveis de entrada em normatizados e originais
X_OLD=[]
X_OLD=X.copy()
X=Normatiza(X)
#print(X)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
print(y.ravel())

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 

In [15]:
# esta parte encontra os indices do indefinido para depois 
# usar os indicies para unir indefinido e buraco

Ind=[i for i, e in enumerate(y) if e == 3]
print(Ind)

[360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539]


In [16]:
# esta parte une indefinido e buraco no numero 1 e passa a ser uma so 
# classe por enquanto os dados nao estao balanceados 
# tem o dobro da classe 1 que a classe 2
for i in Ind:
  y[i]=1
print(y.ravel())

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, 
                                                    random_state=25)

In [0]:
# define a rede neural 
#N_Hidden=(18,18,18)


clf = MLPClassifier(solver='lbfgs',activation=Ativa,alpha=1e-5,
                    hidden_layer_sizes=N_Hidden,random_state=0)


In [19]:
# treina a rede neural
clf.fit(X_train, y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(activation='tanh', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=22, learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=0, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [0]:
# usa a rede neural treina em calculos para treino e teste
y_calc_train=clf.predict(X_train)
y_pred=clf.predict(X_test)


In [21]:
# Evaluating the Algorithm
print(confusion_matrix(y_pred, y_test))
print(classification_report(y_pred, y_test))

[[86 18]
 [18 40]]
              precision    recall  f1-score   support

           1       0.83      0.83      0.83       104
           2       0.69      0.69      0.69        58

    accuracy                           0.78       162
   macro avg       0.76      0.76      0.76       162
weighted avg       0.78      0.78      0.78       162



In [22]:
'''
[[31  5 15]
 [ 8 41 11]
 [12 12 27]]
              precision    recall  f1-score   support

           1       0.61      0.61      0.61        51
           2       0.71      0.68      0.69        60
           3       0.51      0.53      0.52        51

    accuracy                           0.61       162
   macro avg       0.61      0.61      0.61       162
weighted avg       0.61      0.61      0.61       162
'''


'\n[[31  5 15]\n [ 8 41 11]\n [12 12 27]]\n              precision    recall  f1-score   support\n\n           1       0.61      0.61      0.61        51\n           2       0.71      0.68      0.69        60\n           3       0.51      0.53      0.52        51\n\n    accuracy                           0.61       162\n   macro avg       0.61      0.61      0.61       162\nweighted avg       0.61      0.61      0.61       162\n'

In [23]:
print(y_pred)

[1 1 1 2 1 1 1 1 2 1 1 2 1 1 1 2 2 1 2 1 1 1 1 1 2 1 2 1 2 1 1 2 1 2 2 1 2
 1 1 2 1 1 1 1 1 1 2 2 2 2 1 1 1 1 2 1 2 2 1 2 1 2 1 1 1 1 1 1 2 1 1 1 1 2
 2 2 1 2 1 2 1 1 2 2 1 2 2 1 1 2 2 1 1 2 1 1 1 1 1 2 1 1 1 1 1 2 1 2 2 2 1
 1 1 1 1 2 1 2 1 2 1 1 1 1 2 2 1 1 1 1 1 1 1 1 1 1 2 1 1 1 2 1 2 1 1 1 1 2
 2 2 1 1 2 2 1 2 2 2 1 1 1 2]
